# Pain in the Net
Replication of *Deeper Image Quality Transfer: Training Low-Memory Neural Networks for 3D Images*


Code by:

Tyler Spears - tas6hh@virginia.edu

Dr. Tom Fletcher

---

Source work:
`S. B. Blumberg, R. Tanno, I. Kokkinos, and D. C. Alexander, “Deeper Image Quality Transfer: Training Low-Memory Neural Networks for 3D Images,” in Medical Image Computing and Computer Assisted Intervention – MICCAI 2018, Cham, 2018, pp. 118–125, doi: 10.1007/978-3-030-00928-1_14.`


## Imports & Environment Setup

In [1]:
# imports
import math
import random
import itertools
import collections
import functools
import pathlib
from pathlib import Path
import warnings
import sys
import os
import subprocess
import io

import dotenv
# Computation & ML libraries.
import numpy as np
import pandas as pd
import skimage
import skimage.measure, skimage.feature, skimage.filters
import torch
import torch.nn.functional as F
import torchvision
import pytorch_lightning as pl
import nilearn
import nilearn.plotting
import dipy
import dipy.reconst, dipy.reconst.dti, dipy.segment.mask, dipy.core
# Data management libraries.
import nibabel as nib
import torchio
import natsort
from natsort import natsorted

# visualization libraries
%matplotlib inline
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.autolayout': True})
plt.rcParams.update({'figure.facecolor': [1.0, 1.0, 1.0, 1.0]})

# Set print options for ndarrays/tensors.
np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

/opt/miniconda/envs/pitn/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
# Update notebook's environment variables with direnv.
# This requires the python-dotenv package, and direnv be installed on the system
# This will not work on Windows.
# NOTE: This is kind of hacky, and not necessarily safe. Be careful...
# Libraries needed on the python side:
# - os
# - subprocess
# - io
# - dotenv

# Form command to be run in direnv's context. This command will print out
# all environment variables defined in the subprocess/sub-shell.
command = "direnv exec {} /usr/bin/env".format(os.getcwd())
# Run command in a new subprocess.
proc = subprocess.Popen(
    command, stdout=subprocess.PIPE, shell=True, cwd=os.getcwd())
# Store and format the subprocess' output.
proc_out = proc.communicate()[0].strip().decode('utf-8')
# Use python-dotenv to load the environment variables by using the output of
# 'direnv exec ...' as a 'dummy' .env file.
dotenv.load_dotenv(stream=io.StringIO(proc_out), override=True);

In [3]:
# Project-specific scripts
# It's easier to import it this way rather than make an entirely new package, due to
# conflicts with local packages and anaconda installations.
# You made me do this, poor python package management!!
if 'PROJECT_ROOT' in os.environ:
    src_location = str(Path(os.environ['PROJECT_ROOT']).resolve())
else:
    src_location = str(Path("../../").resolve())
sys.path.append(src_location)
import src as pitn

In [4]:
# torch setup

# allow for CUDA usage, if available
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
# keep device as the cpu
# device = torch.device('cpu')
print(device)

cuda


In [5]:
# Watermark
%load_ext watermark
%watermark --author "Tyler Spears" --updated --iso8601  --python --machine --iversions --githash
print("CUDA Version: ", torch.version.cuda)

Author: Tyler Spears

Last updated: 2021-03-23T17:08:57.494621-04:00

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.21.0

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.8.0-3-amd64
Machine     : x86_64
Processor   : 
CPU cores   : 8
Architecture: 64bit

Git hash: d6684905d05ad2893f627bedb46228505baac3b2

torchvision      : 0.2.2
seaborn          : 0.11.1
nilearn          : 0.7.1
pandas           : 1.2.3
dipy             : 1.3.0
pytorch_lightning: 1.2.3
torchio          : 0.18.29
torch            : 1.8.0
sys              : 3.8.8 (default, Feb 24 2021, 21:46:12) 
[GCC 7.3.0]
matplotlib       : 3.3.4
skimage          : 0.18.1
nibabel          : 3.2.1
natsort          : 7.1.1
numpy            : 1.20.1

CUDA Version:  11.1


## Variables & Definitions Setup

In [6]:
# Set up directories
data_dir = pathlib.Path(os.environ['DATA_DIR']) / "hcp"
assert data_dir.exists()
write_data_dir = pathlib.Path(os.environ['WRITE_DATA_DIR']) / "hcp"
assert write_data_dir.exists()
results_dir = pathlib.Path(os.environ['RESULTS_DIR'])
assert results_dir.exists()

### Global Function & Class Definitions

In [7]:
# For more clearly designating the return values of a reader function given to
# the `torchio.Image` object.
ReaderOutput = collections.namedtuple('ReaderOutput', ['dti', 'affine'])
def dti_fit_selection_reader(
    f_dwi,
    idx: np.ndarray,
    bvals: np.ndarray,
    bvecs: np.ndarray,
    brain_mask=None,
    downsample_factor: int = 1
) -> ReaderOutput:
    print("Starting load & dti fit", flush=True)
    # Load entire image into memory, then slice that full image.
    # A float32 is the smallest representation that doesn't lose data.
    img = nib.load(f_dwi)


    if downsample_factor <= 1:
        sliced_img = img.get_fdata()[..., idx]
    else:
        sliced_img = img.get_fdata()[
            ::downsample_factor,
            ::downsample_factor,
            ::downsample_factor,
            idx
        ]

        if brain_mask is not None:
            brain_mask = brain_mask[
                ::downsample_factor,
                ::downsample_factor,
                ::downsample_factor,
            ]
    print("Loaded & sliced image", flush=True)

    gradient_table = dipy.core.gradients.gradient_table_from_bvals_bvecs(
        bvals=bvals,
        bvecs=bvecs,
    )
    print("Created gradient table", flush=True)
    tensor_model = dipy.reconst.dti.TensorModel(gradient_table, fit_method='WLS')
    dti = tensor_model.fit(sliced_img, mask=brain_mask)
    # Pull only the lower-triangular part of the DTI (the non-symmetric coefficients.)
    dti = dti.lower_triangular()
    print("Fitted DTI model", flush=True)

    affine = img.affine.copy()
    # Hopefuly remove any refs to the full-size array.
    del img

    return ReaderOutput(
        dti=torch.from_numpy(dti.astype(np.float32)),
        affine=torch.from_numpy(affine.astype(np.float32))
    )

### Global Parameters

In [8]:
downsample_factor = 2
bvals_range = (0, 1500)

## Data Loading

In [9]:
# Find data directories for each subject.
subj_dirs: dict = dict()


selected_ids = [
    '397154',
    '224022',
    '140117',
    '751348',
    '894774',
    '156637',
    '227432',
    '303624',
    '185947',
    '810439',
    '753251',
    '644246',
    '141422',
    '135528',
    '103010',
    '700634',
]

## Sub-set the chosen participants for dev and debugging!
selected_ids = random.sample(selected_ids, 6)
warnings.warn(
    "WARNING: Sub-selecting participants for dev and debugging. "
    + f"Subj IDs selected: {selected_ids}"
)
##

selected_ids = natsorted(list(map(lambda s: int(s), selected_ids)))

for subj_id in selected_ids:
    subj_dirs[subj_id] = data_dir / f"{subj_id}/T1w/Diffusion"
    assert subj_dirs[subj_id].exists()
subj_dirs

<ipython-input-9-a979ab82375b>:26: UserWarning: WARNING: Sub-selecting participants for dev and debugging. Subj IDs selected: ['135528', '700634', '224022', '141422', '185947', '644246']
  warnings.warn(


{135528: PosixPath('/srv/data/pitn/hcp/135528/T1w/Diffusion'),
 141422: PosixPath('/srv/data/pitn/hcp/141422/T1w/Diffusion'),
 185947: PosixPath('/srv/data/pitn/hcp/185947/T1w/Diffusion'),
 224022: PosixPath('/srv/data/pitn/hcp/224022/T1w/Diffusion'),
 644246: PosixPath('/srv/data/pitn/hcp/644246/T1w/Diffusion'),
 700634: PosixPath('/srv/data/pitn/hcp/700634/T1w/Diffusion')}

The 90 scans are taken from the b=1000 s/mm^2. However, the b=0 shells are still required for fitting the diffusion tensors (DTI's), so those will need to be kept, too.

To find those, sub-select with the $0 < bvals < 1500$, or roughly thereabout. A b-val of $995$ or $1005$ still counts as a b=1000.

In [10]:
# Import all image data into a sequence of `torchio.Subject` objects.
subj_data: dict = dict()

for subj_id, subj_dir in subj_dirs.items():
    # Sub-select volumes with only bvals in a certain range. E.x. bvals <= 1100 mm/s^2, 
    # a.k.a. only the b=0 and b=1000 shells.
    bvals = torch.as_tensor(np.loadtxt(subj_dir/"bvals").astype(int))
    idx_to_keep = (bvals >= bvals_range[0]) & (bvals <= bvals_range[1])
    bvals = bvals[idx_to_keep]
    bvecs = torch.as_tensor(np.loadtxt(subj_dir/"bvecs"))[:, idx_to_keep]
    # Create a custom, partial reader function that will sub-set the full (~288 for HCP) 
    # data points down to only the selected bvals range.
    # This will still be lazily loaded, while not requiring as much memory per subject.

    # grad = torchio.ScalarImage(subj_dir/"grad_dev.nii.gz")
    brain_mask = torchio.LabelMap(subj_dir/"nodif_brain_mask.nii.gz", type=torchio.LABEL)
    # The brain mask is binary.
    brain_mask.set_data(brain_mask.data.bool())

    # Create a partial reader function and torchio.Image for the full-resolution DTI
    # fitting (a.k.a., the ground truths).
    high_res_reader = functools.partial(
        dti_fit_selection_reader,
        idx=idx_to_keep,
        downsample_factor=1,
        bvals=bvals.numpy(),
        # Pass the transpose of the bvecs to fit DIPY's expected (N, 3) shape.
        bvecs=bvecs.numpy().T,
        brain_mask=brain_mask.data.numpy().squeeze()
    )
    dti = torchio.ScalarImage(
        subj_dir/"data.nii.gz",
        bvals=bvals,
        bvecs=bvecs,
        reader=high_res_reader
    )

    # Create a partial reader function and torchio.Image for the low-resolution DTI
    # fitting (a.k.a., the input patches).
    low_res_reader = functools.partial(
        dti_fit_selection_reader,
        idx=idx_to_keep,
        downsample_factor=downsample_factor,
        bvals=bvals.numpy(),
        # Pass the transpose of the bvecs to fit DIPY's expected (N, 3) shape.
        bvecs=bvecs.numpy().T,
        brain_mask=brain_mask.data.numpy().squeeze()
    )
    low_res_dti = torchio.ScalarImage(
        subj_dir/"data.nii.gz",
        bvals=bvals,
        bvecs=bvecs,
        reader=low_res_reader
    )

    subject_dict = torchio.Subject(
        subj_id=subj_id,
        dti=dti,
        low_res_dti=low_res_dti,
#         grad=grad,
        brain_mask=brain_mask
    )
    subj_data[subj_id] = subject_dict


In [11]:
subj_dataset = torchio.SubjectsDataset(list(subj_data.values()), load_getitem=False)

---

In [12]:
subj_dataset[0].dti.data

Starting load & dti fit
Loaded & sliced image
Created gradient table
Fitted DTI model


AttributeError: 'TensorFit' object has no attribute 'astype'

In [13]:
debug

> <ipython-input-7-93efd443801a>(50)dti_fit_selection_reader()
     48 
     49     return ReaderOutput(
---> 50         dti=torch.from_numpy(dti.astype(np.float32)),
     51         affine=torch.from_numpy(affine.astype(np.float32))
     52     )



ipdb>  dti


ipdb>  dir(dti)


['S0_hat', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'ad', 'adc', 'color_fa', 'directions', 'evals', 'evecs', 'fa', 'ga', 'linearity', 'lower_triangular', 'md', 'mode', 'model', 'model_S0', 'model_params', 'odf', 'planarity', 'predict', 'quadratic_form', 'rd', 'shape', 'sphericity', 'trace']


ipdb>  dti.adc


...

ipdb>  dti.lower_triangular()


array([[[[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        ...,

        [[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0.

ipdb>  dti.lower_triangular().shape


(145, 174, 145, 6)


ipdb>  exit


## Model Training

In [ ]:
# Patch parameters
batch_size = 12
# 6 channels for the 6 DTI components
channels = 6

# Input patch parameters
h_in = 11
w_in = 11
d_in = 11
input_patch_shape = (channels, h_in, w_in, d_in)
# Output patch parameters
h_out = downsample_factor * h_in
w_out = downsample_factor * w_in
d_out = downsample_factor * d_in
unshuffled_channels_out = channels * downsample_factor**3
# Output before shuffling
unshuffled_output_patch_shape = (unshuffled_channels_out, h_in, w_in, d_in)
# Output shape after shuffling.
output_patch_shape = (channels, h_out, w_out, d_out)

### Model Definition

In [ ]:
# Define DTI-fitting procedure for pre-processing input and ground-truth.
def fit_dti_patches(dwi_patches):
    model = dipy.reconst.dti.TensorModel(

In [ ]:
# Define pytorch-lightning module.
class DIQTSystem(pl.LightningModule):

    def __init__(self, channels_in, channels_out):
        super().__init__()

        self.channels_in = channels_in
        self.channels_out = channels_out
        # Parameters
        # Network parameters
        self.num_revnet_layers = 4
        self.net = pitn.models.ESPCN_RN(
            no_RevNet_layers=self.num_revnet_layers,
            no_chans_in=self.channels_in,
            no_chans_out=self.channels_out,
            memory_efficient=True
        )

        ## Training parameters
        self.max_epochs = 100
        self.lr = 10e-4
        self.loss = torch.nn.MSELoss(reduction='mean')

    def forward(self, x):
        y = self.net(x)
        return y
    
    def training_step(self, batch, batch_idx):
        pass

    def validation_step(self, batch, batch_idx):
        pass

    def test_step(self, batch, batch_idx):
        pass

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.net.parameters(), lr=self.lr)
        return optimizer

### Set Up Patch-Based Data Loaders

In [ ]:
# Data train/validation/test split
test_percent = 0.2
train_percent = 1 - test_percent
val_percent = 0.1

num_subjs = len(subj_dataset)
num_test_subjs = int(np.ceil(num_subjs * test_percent))
num_train_subjs = num_subjs - num_test_subjs
subj_list = subj_dataset.dry_iter()
# Randomly shuffle the list of subjects, then choose the first `num_test_subjs` subjects
# for testing.
random.shuffle(subj_list)
test_dataset = torchio.SubjectsDataset(subj_list[:num_test_subjs], load_getitem=False)
# Choose the remaining for training/validation.
subj_list = subj_list[num_test_subjs:]
train_dataset = torchio.SubjectsDataset(subj_list, load_getitem=False)

# Training patch sampler, random across all patches of all volumes.
train_sampler = torchio.LabelSampler((h_in, w_in, d_in), 'brain_mask', {0: 0, 1:1})
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    sampler=train_sampler,
    num_workers=4,
    pin_memory=True
)
# Test samplers must be dynamically created during testing.

print("Test subject(s) IDs: ", [s.subj_id for s in test_dataset.dry_iter()])
print("Training subject(s) IDs: ", [s.subj_id for s in train_dataset.dry_iter()])

### Training

In [ ]:
model = DIQTSystem(channels_in=channels, channels_out=unshuffled_channels_out)
trainer = pl.Trainer(gpus=1, progress_bar_refresh_rate=10)
trainer.fit(model, train_loader)

## Model Evaluation

### Testing

### Visualization